<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/NLP%20Course%20-%20Week%203%20Exercise%20Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Blog Binary Classification Prevent Overfitting

In [30]:
import json
import tensorflow as tf
import csv
import random
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers


embedding_dim = 100
max_length = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size=160000
test_portion=.1

corpus = []


In [31]:

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv

num_sentences = 0


#Create array with shape (totalSentence,2) where [][0] is sentence
# and [][1] is the label which is arranged as 0 or 1.

with open("/tmp/training_cleaned.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        list_item=[]
        list_item.append(row[5])
        this_label=row[0]
        if this_label=='0':
            list_item.append(0)
        else:
            list_item.append(1)
        num_sentences = num_sentences + 1
        corpus.append(list_item)


--2020-09-15 18:23:18--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238942690 (228M) [application/octet-stream]
Saving to: ‘/tmp/training_cleaned.csv’

/tmp/training_clean 100%[===================>] 227.87M   141MB/s    in 1.6s    

2020-09-15 18:23:20 (141 MB/s) - ‘/tmp/training_cleaned.csv’ saved [238942690/238942690]



In [32]:
print(num_sentences)
print(len(corpus))
print(corpus[1])

# Expected Output:
# 1600000
# 1600000
# ["is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", 0]

1600000
1600000
["is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", 0]


In [34]:
sentences=[]
labels=[]
random.shuffle(corpus)
for x in range(training_size):
    sentences.append(corpus[x][0])
    labels.append(corpus[x][1])


tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
print("word index : ",word_index)
vocab_size=len(word_index)

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

split = int(test_portion * training_size)

test_sequences = padded[0:split]
training_sequences = padded[split:training_size]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

word index :  {'i': 1, 'to': 2, 'the': 3, 'a': 4, 'my': 5, 'and': 6, 'you': 7, 'is': 8, 'it': 9, 'in': 10, 'for': 11, 'of': 12, 'on': 13, 'me': 14, 'so': 15, 'have': 16, 'that': 17, 'but': 18, "i'm": 19, 'just': 20, 'with': 21, 'be': 22, 'at': 23, 'not': 24, 'was': 25, 'this': 26, 'now': 27, 'good': 28, 'up': 29, 'day': 30, 'get': 31, 'all': 32, 'out': 33, 'like': 34, 'are': 35, 'no': 36, 'quot': 37, 'go': 38, 'http': 39, 'today': 40, 'your': 41, 'do': 42, "it's": 43, 'too': 44, 'going': 45, 'work': 46, 'love': 47, 'got': 48, 'time': 49, 'lol': 50, 'back': 51, 'from': 52, 'what': 53, 'u': 54, 'one': 55, 'com': 56, 'im': 57, 'know': 58, 'will': 59, 'we': 60, 'am': 61, 'really': 62, "don't": 63, 'about': 64, 'amp': 65, 'can': 66, 'had': 67, 'some': 68, 'see': 69, "can't": 70, 'its': 71, 'if': 72, 'night': 73, '2': 74, 'still': 75, 'new': 76, 'how': 77, 'well': 78, 'want': 79, 'think': 80, 'home': 81, 'thanks': 82, 'there': 83, 'when': 84, 'oh': 85, 'as': 86, 'more': 87, 'they': 88, 'here

In [ ]:
print(vocab_size)
print(word_index['i'])

# Expected Output
# 138858
# 1

138738
1


In [35]:
# Note this is the 100 dimension version of GloVe from Stanford
# I unzipped and hosted it on my site to make this notebook easier
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt
embeddings_index = {};
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

--2020-09-15 18:36:04--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M   171MB/s    in 1.9s    

2020-09-15 18:36:07 (171 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [36]:
print(len(embeddings_matrix))
# Expected Output
# 138859

138723


In [37]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()



Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 16, 100)           13872300  
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 100)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 12, 64)            32064     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 3, 64)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 13,937,453
Trainable params: 65,153
Non-trainable params: 13,872,300
_____________________________________

In [38]:
num_epochs = 15

training_padded = np.array(training_sequences)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

print("Training Complete")

Epoch 1/15
4500/4500 - 15s - loss: 0.5685 - accuracy: 0.6977 - val_loss: 0.5276 - val_accuracy: 0.7352
Epoch 2/15
4500/4500 - 16s - loss: 0.5284 - accuracy: 0.7311 - val_loss: 0.5211 - val_accuracy: 0.7365
Epoch 3/15
4500/4500 - 15s - loss: 0.5112 - accuracy: 0.7424 - val_loss: 0.5120 - val_accuracy: 0.7434
Epoch 4/15
4500/4500 - 15s - loss: 0.4996 - accuracy: 0.7500 - val_loss: 0.5041 - val_accuracy: 0.7504
Epoch 5/15
4500/4500 - 15s - loss: 0.4899 - accuracy: 0.7565 - val_loss: 0.4992 - val_accuracy: 0.7563
Epoch 6/15
4500/4500 - 15s - loss: 0.4848 - accuracy: 0.7610 - val_loss: 0.5092 - val_accuracy: 0.7564
Epoch 7/15
4500/4500 - 16s - loss: 0.4774 - accuracy: 0.7655 - val_loss: 0.5095 - val_accuracy: 0.7490
Epoch 8/15
4500/4500 - 15s - loss: 0.4722 - accuracy: 0.7683 - val_loss: 0.4974 - val_accuracy: 0.7581
Epoch 9/15
4500/4500 - 15s - loss: 0.4685 - accuracy: 0.7723 - val_loss: 0.5017 - val_accuracy: 0.7586
Epoch 10/15
4500/4500 - 15s - loss: 0.4628 - accuracy: 0.7753 - val_loss:

# Without transferring Embedding Layer


In [ ]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])

plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])

plt.figure()


# Expected Output
# A chart where the validation loss does not increase sharply!